Para poder entrenar nuestros modelos de machine learning debemos crear un archivo de ingeniería de características.  



Text Cleaning and Preparation: cleaning of special characters, downcasing, punctuation signs. possessive pronouns and stop words removal and lemmatization.
Label coding: creation of a dictionary to map each category to a code.
Train-test split: to test the models on unseen data.
Text representation: use of TF-IDF scores to represent text.

# Importación de las librerías

In [1]:
import pandas as pd
import seaborn as sns
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from emoji import UNICODE_EMOJI
from nltk import bigrams 


# Lectura de la muestra

In [2]:
df = pd.read_csv("C:/Users/Esperanza/Desktop/UOC/TFM/python/00_Dataset/clean_sentiment_labels_completo.csv", sep=';')
df.head()
df.is_copy = False

C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\generic.py:4384: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\generic.py:4385: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


# Limpieza de los datos
Una limpieza de los datos ha sido realizada tanto en el EDA como en la extracción. Sin embargo, aún hay algunos detalles que debemos depurar para poder hacer un buen análisis sobre nuestros datos.  

Es  habitual la supresión de signos de puntuación pero en este caso nos ayudan a identificar la intensidad de los comentarios.

**Supresión de las columnas no necesarias***

In [3]:
df_eng=df[['texto', 'num_may_esc','num_insultos','score_emoji_tox','emphasize','toxico']]
df_eng.head()

,texto,num_may_esc,num_insultos,score_emoji_tox,emphasize,toxico
0,"La etiqueta, cabrón",0.0,1,0.0,0.0,1
1,Hasta que un cabron trata a la lacra cómo se d...,0.0,1,0.0,0.0,1
2,La edad no la pones eh cabrón,0.0,1,0.0,1.0,1
3,Y de un cabron solo puedes obtener cabronadas,0.0,1,0.0,1.0,1
4,Eres más guiri que los irlandeses de aquí cabr...,0.1,1,0.0,1.1,1


**Supresión de abreviaturas en el texto y conservión a minúsculas**

In [4]:
#Creamos un diccionario desde el csv que hemos creado
path_ab='C:/Users/Esperanza/Desktop/UOC/TFM/python/00_Dataset/abreviaturas.csv'
abrev = pd.read_csv(path_ab, sep=';')
diccionario_abr=abrev.set_index('abreviatura').T.to_dict('dict')

In [5]:
#Creación de una función en el que palabras como xa se convierten en para
def delete_abrev(string):
    words=string.split()
    for i in range(len(words)):
        if words[i] in diccionario_abr.keys():
            words[i] = diccionario_abr[words[i]]['significado'].lower()
    return " ".join(words)



#Comprobación del funcionamiento
aux=df_eng.iloc[48]['texto']
print(aux)
delete_abrev(aux)

una parguela que es famosa xq es influenser


'una parguela que es famosa porque es influenser'

In [6]:
#Sustitución de las abreviaciones en el dataset original
df_eng.loc[:,'texto']=df_eng.loc[:,'texto'].map(lambda x: delete_abrev(x))
df_eng.is_copy = False
df_eng.head()

C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\generic.py:4384: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
C:\Users\Esperanza\Anaconda3\lib\site-packages\pandas\core\generic.py:4385: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


,texto,num_may_esc,num_insultos,score_emoji_tox,emphasize,toxico
0,"La etiqueta, cabrón",0.0,1,0.0,0.0,1
1,Hasta que un cabron trata a la lacra cómo se d...,0.0,1,0.0,0.0,1
2,La edad no la pones eh cabrón,0.0,1,0.0,1.0,1
3,Y de un cabron solo puedes obtener cabronadas,0.0,1,0.0,1.0,1
4,Eres más guiri que los irlandeses de aquí cabr...,0.1,1,0.0,1.1,1


**Tokenización quitando las stopwords y los signos de puntuación**

No siempre se han de quitar todas las stopwords ya que algunas como no si tienen peso en el algoritmo. En este caso eliminamos no del conjunto de palabras vacías y añadimos los signos de puntuación por tenerlos ya en cuenta en la etiqueta emphasize. 

In [7]:
stopWords = stopwords.words('spanish')
stopWords.remove('no')

df_eng.loc[:,'filtered_words']=df_eng.loc[:,'texto'].map(lambda texto: texto.replace('.',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace('?',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace('!',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace('¿',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace(',',''))
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda texto: texto.replace('“',''))
#Supresión de las stopword en castellano
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].apply(lambda frase: [word.lower() for word in frase.split(' ') if word.lower() not in stopWords])

df_eng.loc[6,'filtered_words']


['cabron', 'problema', 'solucion']

**Lemmanization**

Lemmatización es el proceso de convertir una palabra a su forma básica. La diferencia entre stemming y lemmatization es que la lematización considera el contexto y convierte la palabra a su forma básica significativa, mientras que stemming simplemente elimina los últimos caracteres, lo que a menudo conduce a significados incorrectos y errores de ortografía.

fuente: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [8]:
#Prueba de funcionamiento
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("spanish")
stemmer.stem("estudiando")

'estudi'

In [9]:
import spacy
nlp = spacy.load('es_core_news_sm')
text1 = "Estoy estudiando como funciona lemma en español y como no funciona.Pero que dices zampabollos "
res=[]
for token in nlp(text1):
    res.append(token.lemma_)
print(res)

['Estoy', 'estudiar', 'comer', 'funcionar', 'lemma', 'en', 'español', 'y', 'comer', 'no', 'funcionar', '.', 'Pero', 'que', 'decir', 'zampabollos']


In [10]:
#Definición de la función que extrae el origen de la palabra
def extraccion_lemma(text): 
    str1 = ' '.join(str(e) for e in text)
    res=[]
    for word in nlp(str1):
        res.append(word.lemma_)
    return res



Podemos ver que la raiz no es tomada correctamente y no proporciona valor. Pero el lemma funciona suficientemente bien en castellano.  

Algunos bugs como los presentados en https://github.com/explosion/spaCy/issues/2710 siguen sin ser solucionados. Palabras como 'estoy' no son traducidas a estar, 'como' es traducido por comer,.... 

In [11]:
df_eng.loc[:,'filtered_words']=df_eng.loc[:,'filtered_words'].map(lambda text: extraccion_lemma(text) )
df_eng.head()

,texto,num_may_esc,num_insultos,score_emoji_tox,emphasize,toxico,filtered_words
0,"La etiqueta, cabrón",0.0,1,0.0,0.0,1,"[etiquetar, cabrón]"
1,Hasta que un cabron trata a la lacra cómo se d...,0.0,1,0.0,0.0,1,"[cabron, tratar, lacrar, cómo, deber, indignar]"
2,La edad no la pones eh cabrón,0.0,1,0.0,1.0,1,"[edad, no, poner, eh, cabrón]"
3,Y de un cabron solo puedes obtener cabronadas,0.0,1,0.0,1.0,1,"[cabron, solo, poder, obtener, cabronada]"
4,Eres más guiri que los irlandeses de aquí cabr...,0.1,1,0.0,1.1,1,"[guiri, irlandés, aquí, cabrón, xd]"


# Dividiendo el dataset en train, dev and test sets 
Pasos a tener en cuenta: 
- dev y test deben proceder de la misma distribución y se debe tomar de forma aleatoria del total del corpus. 
- dev y test deben tener el mismo tamaño. En nuestro caso dividiremos en train 70%, dev 15% y test 15%.
- Se quiere tener unos datos no sesgados a la hora de realizar el algoritmo, para ello se cambia el orden de las filas pero inicializando una semilla que fije los resultados dependientes de una variable aleatoria. 


fuente: https://www.coursera.org/learn/machine-learning-projects

In [12]:
dfmod=df_eng.reindex(np.random.RandomState(seed=42).permutation(df_eng.index))
dfmod.head()

,texto,num_may_esc,num_insultos,score_emoji_tox,emphasize,toxico,filtered_words
646,Esperemos que el Big Data de el visto bueno pa...,0.0,0,0.0,0.0,0,"[esperar, big, datar, vestir, bueno, pagar, cl..."
336,Este pobre payaso ya no sabe donde esta el nor...,0.0,0,0.0,1.0,1,"[pobre, payaso, no, saber, norte, sur]"
63,"MB98 Seguro que y luego unas risas, esas respu...",0.0,0,0.0,1.0,1,"[mb98, seguro, luego, uno, risa, respuesta, gi..."
367,"Sigan peleando, la gente de a pie esta jodida ...",0.0,1,0.0,1.0,1,"[seguir, pelear, gente, pie, joder, desacelera..."
101,"Casa Rural en Sariñena, Los Monegros, en Huesc...",0.0,0,0.0,1.0,0,"[casar, rural, sariñena, monegros, huesca, 12,..."


In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(dfmod['filtered_words'], dfmod['toxico'], test_size=0.2, random_state=42)
#everytime you run it without specifying random_state, you will get a different result
print(x_train.shape, y_train.shape) 
print( x_test.shape, y_test.shape)


(542,) (542,)
(136,) (136,)


# Salvamento de los datos

Pickle es utilizado cuando se tiene un conjunto de datos grande y se carga ese conjunto de datos masivos en la memoria cada vez que ejecuta el programa.Almacenarlo y recuperarlo será mucho más rápido, del orden de 50 - 100x, a veces mucho más dependiendo del tamaño.

fuente: https://pythonprogramming.net/python-pickle-module-save-objects-serialization/


In [ ]:
import pickle

In [ ]:
#Almacenamiento del df del modelo
pickle_out = open("dfmodelo.pickle","wb")
pickle.dump(dfmod, pickle_out)
pickle_out.close()

In [ ]:
#Almacenamiento del x_train
pickle_out = open("x_train.pickle","wb")
pickle.dump(x_train, pickle_out)
pickle_out.close()

In [ ]:
#Almacenamiento del x_test
pickle_out = open("x_test.pickle","wb")
pickle.dump(x_test, pickle_out)
pickle_out.close()

In [ ]:
#Almacenamiento del y_train
pickle_out = open("y_train.pickle","wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

In [ ]:
#Almacenamiento del y_test
pickle_out = open("y_test.pickle","wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()